In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import time


def setGraph( N, edgeList ):
    W = np.zeros((N,N))
    for e in edgeList:
        i = e[0]
        j = e[1]
        W[i][j] = 1
        W[j][i] = 1       
    return W

def setIncidence( N, edgeList ):
    m = len(edgeList)
    B = np.zeros((N,m))
    for idx, e in enumerate(edgeList):
        i = e[0]
        j = e[1]
        B[i,idx] = 1
        B[j,idx] = -1
    return B

def drawLines(k, edgeList, x, x_inf, y):
    plt.cla()
    #size = 20
    #ax3.set_xlim3d(0, size)
    #ax3.set_ylim3d(0, size)
    #ax3.set_zlim3d(0, size)
    for i in range(N):
        #ax3.scatter(x[3*i,0],x[3*i+1,0],x[3*i+2,0],c='blue') # initial estimated position
        ax3.scatter(y[3*i,0],y[3*i+1,0],y[3*i+2,0],c='red') # initial measured position
        #ax3.scatter(x[3*i,K-1],x[3*i+1,K-1],x[3*i+2,K-1],c='blue') # final estimated position
        #ax3.scatter(y[3*i,K-1],y[3*i+1,K-1],y[3*i+2,K-1],c='red') # final measured position
        ax3.scatter(x_inf[3*i],x_inf[3*i+1],x_inf[3*i+2],c='green') # desired position
        
        #ax3.scatter(x[3*i,k-1],x[3*i+1,k-1],x[3*i+2,k-1],c='blue') # estimated position
        ax3.scatter(y[3*i,k-1],y[3*i+1,k-1],y[3*i+2,k-1],c='red') # measured position
        
        #plt.plot(x[3*i,:],x[3*i+1,:],x[3*i+2,:])   # plot estimated trajectories of each quad
        plt.plot(y[3*i,:],y[3*i+1,:],y[3*i+2,:])   # plot real trajectories of each quad
    for e in edgeList:
        i = e[0]
        j = e[1]
        ax3.plot([x[3*i,k-1], x[3*j,k-1]], [x[3*i+1,k-1], x[3*j+1,k-1]], [x[3*i+2,k-1], x[3*j+2,k-1]], c='black')
    
if __name__ == '__main__':
    n = 3 # dimention where robots live
    N = 8 # number of robots
    
    K = 2000 # number of simulation iteration
    dt = 0.02 # discretization contants
    if N == 8:
        edgeList = [(0,1),
                    (1,2),
                    (2,3),
                    (3,0),
                    (4,5),
                    (5,6),
                    (6,7),
                    (7,4),
                    (0,4),
                    (1,5),
                    (2,6),
                    (3,7)]
    else:
        edgeList = [(0,1),
                    (1,2),
                    (2,3)]        
    W = setGraph(N, edgeList)
    Dout = np.diag(np.sum(W, axis=1))
    L = Dout - W
    if N == 8:
        G_pos = np.diag(np.array([1, 1, 1, 1, 1, 1, 1, 1])) # list of nodes that have position estimates
        G_vel = np.diag(np.array([1, 1, 1, 1, 1, 1, 1, 1])) # list of nodes that have velocity estimates
    else:
        G_pos = np.diag(np.array([1, 0, 0, 0])) # list of nodes that have position estimates
        G_vel = np.diag(np.array([1, 1, 1, 1])) # list of nodes that have velocity estimates        
    B = setIncidence(N, edgeList)
    In = np.identity(n)
    
    idx_ap = [0,1,2,3,4] # list of nodes that have absolute position measurements
    E = np.identity(N)[:, idx_ap] # selection matrix for absolute position measurements
    H = np.concatenate((B.T, E.T), axis=0) # observation matrix           
    
    #----------------------------controller---------------------
    # gains
    krp = 0.4
    krv = 2.0*krp
    kap = 1*krp
    kav = 3*krp    
    # closed loop controller dynamics matrices
    A11 = np.zeros((n*N,n*N))
    A12 = np.identity(n*N)
    A1 = np.concatenate((A11, A12), axis=1)
    
    A21 = -krp*np.kron(L, np.identity(n)) - kap*np.kron(G_pos, np.identity(n))
    A22 = -krv*np.kron(L, np.identity(n)) - kav*np.kron(G_vel, np.identity(n))
    A2 = np.concatenate((A21,A22), axis=1)
    
    A_CTRL = dt*np.concatenate((A1,A2), axis=0)
    print(np.max(np.absolute(np.linalg.eigvals(A_CTRL))))# print norm of largest eigenvalues of error dynamic matrix
    #print(np.linalg.eigvals(A_CTRL))
    
    #----------------------------estimator----------------------
    # gains
    kalman_p = 0.9
    kalman_v = 4*kalman_p
    kalman_abs = 1
    kalman_rel = kalman_abs
    
    # estimator matrix
    #_B = np.dot(np.diag(1/np.sum(np.abs(B),axis=1)),B)
    _K_kalp = np.concatenate((kalman_rel*B, kalman_abs*E), axis=1)
    _K_kalp = np.dot(np.diag(1/np.sum(np.abs(_K_kalp),axis=1)),_K_kalp)
    K_kalp = np.kron(_K_kalp, In)
    K_kal = [kalman_p*K_kalp, 
             kalman_v*K_kalp]
    

    # estimator error dynamics matrices
    # \dot error(t) = MM * error(t)
    A_EST = np.kron(np.kron(np.array([[0, 1],[0, 0]]), np.identity(N)), np.identity(n))
    H_EST = np.kron(np.concatenate((H, np.zeros((H.shape[0],N))), axis=1), np.identity(n))
    K_EST = np.concatenate((K_kal[0],K_kal[1]), axis=0)
    M_EST = A_EST - np.dot(K_EST, H_EST)    
    
    print(np.max(np.real(np.linalg.eigvals(M_EST)))) # print largest negative real parts of eigenvalues of error dynamic matrix
    #print(np.linalg.eigvals(np.dot(K_kal[0],np.kron(H,np.identity(n))))/2)
    
    
    
    #---------------------------Initilization---------------------
    p = np.zeros((n*N, K))
    if N == 8:
        p_inf = np.array([0,0,0, 0,1,0, 1,1,0, 1,0,0, 0,0,1, 0,1,1, 1,1,1, 1,0,1]).T+2.0
    else:
        p_inf = np.array([0,0,0, 0,1,0, 1,1,0, 1,0,0]).T
    p[:,0] = 5*(np.random.rand(n*N)-0.5) # p_inf
    p_est = np.zeros((n*N, K))
    p_est[:,0] = p[:,0]
    
    v = np.zeros((n*N, K))
    v_inf = np.zeros(n*N).T
    v[:,0] = np.kron(np.zeros(N), np.zeros(n)).T
    v_est = np.zeros((n*N, K))
    v_est[:,0] = np.kron(np.zeros(N), np.zeros(n)).T
    
    x = np.zeros((2*n*N,K)) # state vector [p', v']'
    y = np.zeros((n*N,K)) # absolute position measurement, the relative measurment is calculated from it
    y[:,0] = p[:,0]
    z = np.zeros((n*H.shape[0],K)) # relative position measurement
    


In [27]:
    # SIMULATION
    for k in range(K-1):
        #p_inf = p_inf + 0.1*np.sin(k*(np.pi)/360)
        #if k==800:
            #p_inf = p_inf - 0.5*(p_inf-np.kron(np.mean(np.reshape(p_inf,(8,3)), axis=0), np.ones(8)))
        # real dynamics
        u = np.dot(A21, p_est[:,k]-p_inf) + np.dot(A22, v_est[:,k]-v_inf)
        print("original u")
        print(u)
        # input constraints
        u = 0.5*np.tanh(u/0.5)
        print("constrainted u")
        print(u)
        p[:,k+1] = p[:,k] + dt*v[:,k]
        v[:,k+1] = v[:,k] + dt*u + 0.01*(np.random.rand(n*N)-0.5)
        
        # observation
        y[:,k+1] = p[:,k+1] + 0.01*(np.random.rand(n*N)-0.5)
        z[:,k+1] = np.dot(np.kron(H,In), y[:,k+1])
        
        # kalman filtering
        error = z[:,k+1] - np.dot(np.kron(H,In), p_est[:,k] + dt*v_est[:,k])
        p_est[:,k+1] = p_est[:,k] + dt*v_est[:,k] + np.dot(K_kal[0], error)
        v_est[:,k+1] = v_est[:,k] + dt*u*0 + np.dot(K_kal[1], error)

       
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111)
    d = 2 # dimension 0:x, 1:y, z:2
    ax1.plot(range(K),y[0*n+d,:],range(K),y[1*n+d,:],range(K),y[2*n+d,:])
    ax1.plot(range(K),p_est[0*n+d,:],'--',range(K),p_est[1*n+d,:],'--',range(K),p_est[2*n+d,:],'--')
    
    fig2 = plt.figure()
    ax2 = fig2.add_subplot(111)
    d = 2 # dimension 0:x, 1:y, z:2
    ax2.plot(range(K),v[0*n+d,:],'r',range(K),v[1*n+d,:],'g',range(K),v[2*n+d,:],'b')
    ax2.plot(range(K),v_est[0*n+d,:],'r--',range(K),v_est[1*n+d,:],'g--',range(K),v_est[2*n+d,:],'b--')
    
    fig3 = plt.figure()
    ax3 = fig3.add_subplot(111, projection='3d')

    #ax2.scatter(p_est[6,K-1],p_est[7,K-1],p_est[8,K-1])
    ani = animation.FuncAnimation(fig3, drawLines, fargs=[edgeList, p_est, p_inf, y], 
                                  frames=np.arange(1,K-1,10), interval=1)   
    
    plt.show()    


original u
[ 4.3015315   2.55411271 -4.51429127 -1.78669123  0.26131705  1.73760144
  0.01212624  1.37581169 -3.85413941  0.15687464 -0.58517778  4.62332821
 -1.02325863 -1.02311022  2.16249646  0.44788654  4.27555956  5.26894408
  0.87493913 -1.59893667 -0.68602911  0.32369958  1.77604091  4.31549472]
constrainted u
[ 0.49999997  0.49996344 -0.49999999 -0.49921321  0.23986523  0.49904267
  0.01212387  0.495943   -0.4999998   0.15192205 -0.41219306  0.49999999
 -0.48358542 -0.48357583  0.4998249   0.35711674  0.49999996  0.5
  0.47068084 -0.49833414 -0.43958021  0.28495825  0.499179    0.49999997]
original u
[ 4.30289099  2.52667008 -4.42992787 -1.82863618  0.26173902  1.72574252
  0.04118072  1.34338671 -3.83245791  0.12855379 -0.50191344  4.56255914
 -1.07150192 -0.93178682  2.07355006  0.51455932  4.20809175  5.30281315
  0.80055957 -1.48928673 -0.77426956  0.4099015   1.62317483  4.43383336]
constrainted u
[ 0.49999997  0.4999592  -0.49999998 -0.49933466  0.24018996  0.49899621
  0

In [248]:
np.diag(G_pos).T

array([1, 1, 0, 0, 0, 1, 0, 0])

In [71]:
np.mean(np.reshape(p_inf-2,(8,3)), axis=0)

array([ 0.5,  0.5,  0.5])

In [257]:
(np.diag(G_pos)==1)

array([ True,  True, False, False, False,  True, False, False], dtype=bool)

In [55]:
_B

array([[ 1. ,  0. ,  0. ],
       [-0.5,  0.5,  0. ],
       [ 0. , -0.5,  0.5],
       [ 0. ,  0. , -1. ]])

In [24]:
u

array([-0.02729355,  0.02163337, -0.00940705, -0.0007943 ,  0.03876468,
        0.04508984,  0.00241136, -0.02353717, -0.01900441,  0.00216387,
       -0.00013908,  0.01433231, -0.00356161, -0.00882973, -0.00401153,
       -0.03185682,  0.01591871, -0.02425514,  0.01659533,  0.00099878,
        0.00043481, -0.02370086,  0.01677752, -0.02943413])

In [55]:
O=np.kron(np.array([[0,1],[0,0]]),np.identity(n*N))-0.1*np.dot(np.concatenate(np.kron((H.T,0.5*H.T),np.identity(n)),axis=0),H_EST)

In [57]:
np.linalg.eigvals(O)

array([-0.30871150+0.4619618j , -0.30871150-0.4619618j ,
       -0.30871150+0.4619618j , -0.30871150-0.4619618j ,
       -0.30871150+0.4619618j , -0.30871150-0.4619618j ,
       -0.00475623+0.06880122j, -0.00475623-0.06880122j,
       -0.00475623+0.06880122j, -0.00475623-0.06880122j,
       -0.00475623+0.06880122j, -0.00475623-0.06880122j,
       -0.22093304+0.41487544j, -0.22093304-0.41487544j,
       -0.22093304+0.41487544j, -0.22093304-0.41487544j,
       -0.22093304+0.41487544j, -0.22093304-0.41487544j,
       -0.11559924+0.31974373j, -0.11559924-0.31974373j,
       -0.11559924+0.31974373j, -0.11559924-0.31974373j,
       -0.11559924+0.31974373j, -0.11559924-0.31974373j,
       -0.20000000+0.4j       , -0.20000000-0.4j       ,
       -0.20000000+0.4j       , -0.20000000-0.4j       ,
       -0.20000000+0.4j       , -0.20000000-0.4j       ,
       -0.20000000+0.4j       , -0.20000000-0.4j       ,
       -0.20000000+0.4j       , -0.20000000-0.4j       ,
       -0.20000000+0.4j       ,

In [59]:
np.linalg.eigvals(0.1*np.dot(H.T,H))/2

array([ 0.3087115 ,  0.22093304,  0.00475623,  0.11559924,  0.2       ,
        0.1       ,  0.2       ,  0.1       ])

In [45]:
H_EST.shape

(39, 48)

In [37]:
_B

array([[ 1. ,  0. ,  0. ],
       [-0.5,  0.5,  0. ],
       [ 0. , -0.5,  0.5],
       [ 0. ,  0. , -1. ]])

In [27]:
B

array([[ 1.,  0.,  0.],
       [-1.,  1.,  0.],
       [ 0., -1.,  1.],
       [ 0.,  0., -1.]])

In [30]:
np.dot(np.diag(1/np.sum(np.abs(B),axis=1)),B)

array([[ 1. ,  0. ,  0. ],
       [-0.5,  0.5,  0. ],
       [ 0. , -0.5,  0.5],
       [ 0. ,  0. , -1. ]])